In [1]:
import tensorflow as tf
import easydict
import numpy as np
from PIL import Image
import os

2022-04-25 10:07:17.351359: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib64/:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/extras/CUPTI/lib64:
2022-04-25 10:07:17.351394: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def load_pb(path_to_pb):
    """Load the .pb file and return graph

    :param path_to_pb: The path of the .pb file
    :type path_to_pb: str
    :return: graph
    :rtype: GraphDef
    """
    with tf.compat.v1.gfile.GFile(path_to_pb, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.compat.v1.Graph().as_default() as graph:
        tf.compat.v1.import_graph_def(graph_def, name='')
        return graph, graph_def

def analyze_inputs_outputs(graph):
    """Analyzing the graph and return the input node and output node

    :param graph: The computation graph
    :type graph: GraphDef
    :return: [Input Node List, Output Node List]
    :rtype: List
    """
    ops = graph.get_operations()
    outputs_set = set(ops)
    inputs = []
    for op in ops:
        if len(op.inputs) == 0 and op.type != 'Const':
            inputs.append(op)
        else:
            for input_tensor in op.inputs:
                if input_tensor.op in outputs_set:
                    outputs_set.remove(input_tensor.op)
    outputs = list(outputs_set)
    return (inputs, outputs)


def getRandomImage():
    imgDir = "/opt/Anonymous/workspace/mob-dl-rev/data/external/adv_test_imgs/hotdogs"
    length = len(os.listdir(imgDir))
    idx = np.random.randint(low=0, high=length)
    img = Image.open("{}/{}".format(imgDir, os.listdir(imgDir)[idx]))
    # img.show()
    return img

In [ ]:
path = "/opt/Anonymous/workspace/mob-dl-rev/data/raw/tensorflow_model/com.seefoodtechnologies.nothotdog_2017-06-23/deepdog.pb"
graph, graph_def = load_pb(path)
input_nodes, output_nodes = analyze_inputs_outputs(graph)
print("Input Number: {}".format(len(input_nodes)))
print("Output Number: {}".format(len(output_nodes)))
for in_node in input_nodes:
    print("Input Node Name: " + in_node.name)
for out_node in output_nodes:
    print("Out Node Name: " + out_node.name)
input_tensor = graph.get_tensor_by_name('{}:0'.format(input_nodes[0].name))
output_tensor = graph.get_tensor_by_name('{}:0'.format(output_nodes[0].name))

In [ ]:
from IPython.display import display
img = getRandomImage()
img = img.resize(size=(224, 224))
display(img)
img = np.asarray(img).astype(np.float32)
x = img.reshape(1, 224, 224, 3)
x -= 128.
x /= 128.
print(np.min(x), np.max(x))

In [ ]:
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session(graph=graph)
tf.compat.v1.import_graph_def(graph_def)
# input_image = np.random.random(size=(1,224,224,3))
# target = np.random.random(size=(1,1))
# mse_loss = tf.compat.v1.losses.mean_squared_error(output_tensor, target)
# grad = tf.compat.v1.gradients(mse_loss, input_tensor)
out = sess.run(output_tensor, feed_dict={input_tensor: x})
print(out)

In [3]:
# print(np.sum([np.prod(v.get_shape().as_list()) for v in tf.compat.v1.trainable_variables()]))
paths = [
# "com.seefoodtechnologies.nothotdog_2017-06-23/deepdog.pb",
# "uk.tensorzoom_2018-01-18/lv2.pb",
# "uk.tensorzoom_2018-01-18/tz6-s-stitch-sblur-lowtv-gen.pb",
# "uk.tensorzoom_2018-01-18/5-g2-face-fix.pb",
# "uk.tensorzoom_2018-01-18/tz6-s-stitch-blur-gen.pb",
# "uk.tensorzoom_2018-01-18/tz6-s-stitch-gen.pb",
# "uk.tensorzoom_2018-01-18/tensorzoom6-gen.pb",
# "uk.tensorzoom_2018-01-18/tz6-s-stitch-sblur-gen.pb",
# "uk.tensorzoom_2018-01-18/tz6-l-stitch-gen.pb",
# "apk-ch.zhaw.facerecognition/optimized_facenet.pb",
# "com.peace.SilentCamera/people_hq_384x384_1_1543961960_optimized.pb",
# "demo_ssd_mobilenet_v2/ssd_mobilenet_v2_coco_graph.pb",
# "demo_ssd_mobilenet_v1_coco/ssd_mobilenet_v1_coco_graph.pb",
# "demo_mask_rcnn_inception_v2/mask_rcnn_inception_v2_graph.pb",
# "demo_faster_rcnn_inception_v2_coco/faster_rcnn_inception_v2_coco_graph.pb",
# "co.vero.app_2019-07-12/frozen_open_nsfw.pb",
# "com.homesecurity.firstone_2018-06-15/graph.pb",
# "com.lisa.vibe.camera/cartoon_data.pb",
# "apk-com.thmobile.sketchphotomaker/stylize_quantized.pb",
# "pip.face.selfie.beauty.camera.photo.editor_2018-10-29/stylize_quantized.pb",
# "org.tensorflow.detect_2017-10-09/ssd_mobilenet_v1_android_export.pb",
# "Object_Detector_v1.2.2_apkpure.com/ssd_mobilenet_v1_coco.pb",
# "com.app.chakras_opening/ssd_mobilenet_v1_coco.pb",
# "hien.newsvoicereader/ssd_mobilenet_v1_coco.pb",
# "org.tensorflow.detect_2017-10-09/tensorflow_inception_graph.pb",
# "machinelearning.tensorflow.classifier_2017-09-12/tensorflow_inception_graph.pb",
# "org.tensorflow.detect_2017-10-09/conv_actions_frozen.pb",
# "Object_Detector_v1.2.2_apkpure.com/mobilenet_frozen.pb",
# "com.app.chakras_opening/mobilenet_frozen.pb",
# "hien.newsvoicereader/mobilenet_frozen.pb",
# "com.tsinghuabigdata.edu.ddmath_554/answer_judge_symbol.pb",
# "com.steam.photoeditor_2019-07-04/224_1_1_optimized_3.pb",
# "com.shutterfly_2019-07-10/frozen_model_e.pb",
# "com.nhn.android.search_2019-09-18/mobilenet_m27.pb",
# "com.kwalee.drawit_2019-07-19/quickdraw_frozen_long_blacklist_strip_transformed.pb",
# "cn.runagain.run_80/motionDetecter.pb",
# "ai.fritz.heartbeat_2019-04-25/optimized_inference_graph.pb",
# "ai.fritz.heartbeat_2019-04-25/mobilenet_v2_1_0_224_frozen.pb",
# "ai.fritz.heartbeat_2019-04-25/multi_person_mobilenet_v1_075_float_optimized.pb",
# "ai.fritz.heartbeat_2019-04-25/tile_512x512_025_coherent_optimized.pb",
# "ai.fritz.heartbeat_2019-04-25/bicentennial_print_512x512_025_coherent_optimized.pb",
# "ai.fritz.heartbeat_2019-04-25/pink_blue_rhombus_512x512_025_coherent_optimized.pb",
# "apk-faceapp.facemystery.learnmoreaboutyourself/palm_recog_224_1_1.pb",
# "apk-com.optimaxInvestments.GlassesUSA/mtcnn_freezed_model.pb"
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/dh9uvyqf.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/abovapkl_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/i7fnh5oq_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/w4jynptk_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/nhd8kloo.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/7amdhcxt.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/jefetsvp_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/uu7sa93u.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/l8vcxbx2.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/art_jp_comic_1.0.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/art_decoration_1.0.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/qiomib2w.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/ocnzc9mp.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/art_tattoo_1.0_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/rnkhybsi.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/sl2umxmv.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/art_picasso_1.0_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/nnamfco3_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/nq257mmc.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/sxnl51wo.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/nobomm6c.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/dnazi3sf.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/spdafynv_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/to94ylj1.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/h2t5d5ib_025.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/h9t4iyhi.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/yvo67bi4.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/xsnzikde.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/oeyvgadt.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/art_us_comic_1.0.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/fcytc5tg.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/wqqapkz3.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/dra0lg9z.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/wlu2atwy.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/rinwzwg2.pb",
# "com.Video_Mobile_VM.Barnamaj_Damj_Sowar/nus3rdyi_025.pb",
# "beauty.camera.photo.editor/dh9uvyqf.pb",
# "beauty.camera.photo.editor/abovapkl_025.pb",
# "beauty.camera.photo.editor/i7fnh5oq_025.pb",
# "beauty.camera.photo.editor/w4jynptk_025.pb",
# "beauty.camera.photo.editor/nhd8kloo.pb",
# "beauty.camera.photo.editor/7amdhcxt.pb",
# "beauty.camera.photo.editor/jefetsvp_025.pb",
# "beauty.camera.photo.editor/uu7sa93u.pb",
# "beauty.camera.photo.editor/l8vcxbx2.pb",
# "beauty.camera.photo.editor/art_jp_comic_1.0.pb",
# "beauty.camera.photo.editor/art_decoration_1.0.pb",
# "beauty.camera.photo.editor/qiomib2w.pb",
# "beauty.camera.photo.editor/ocnzc9mp.pb",
# "beauty.camera.photo.editor/art_tattoo_1.0_025.pb",
# "beauty.camera.photo.editor/rnkhybsi.pb",
# "beauty.camera.photo.editor/sl2umxmv.pb",
# "beauty.camera.photo.editor/art_picasso_1.0_025.pb",
# "beauty.camera.photo.editor/nnamfco3_025.pb",
# "beauty.camera.photo.editor/nq257mmc.pb",
# "beauty.camera.photo.editor/sxnl51wo.pb",
# "beauty.camera.photo.editor/nobomm6c.pb",
# "beauty.camera.photo.editor/dnazi3sf.pb",
# "beauty.camera.photo.editor/spdafynv_025.pb",
# "beauty.camera.photo.editor/to94ylj1.pb",
# "beauty.camera.photo.editor/h2t5d5ib_025.pb",
# "beauty.camera.photo.editor/h9t4iyhi.pb",
# "beauty.camera.photo.editor/yvo67bi4.pb",
# "beauty.camera.photo.editor/xsnzikde.pb",
# "beauty.camera.photo.editor/oeyvgadt.pb",
# "beauty.camera.photo.editor/art_us_comic_1.0.pb",
# "beauty.camera.photo.editor/fcytc5tg.pb",
# "beauty.camera.photo.editor/wqqapkz3.pb",
# "beauty.camera.photo.editor/dra0lg9z.pb",
# "beauty.camera.photo.editor/wlu2atwy.pb",
# "beauty.camera.photo.editor/rinwzwg2.pb",
# "beauty.camera.photo.editor/nus3rdyi_025.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/best1-v2-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/alpha1-v0-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/best3-v4-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/alpha3-v0-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/health1-v8-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/best2-v3-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/best0-v1-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/ware1-v5-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/super1-v6-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/sketch1-v3-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/pencil1-v7-square.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/pencil1-v7-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/alpha1-v0-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/alpha3-v0-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/super1-v6-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/health1-v8-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/best3-v4-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/best1-v2-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/sketch1-v3-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/ware1-v5-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/best0-v1-rect.pb",
# "apk-com.infixtools.sketchphotomaker.pencil.sketch/best2-v3-rect.pb",
"com.moment.freedomspeak/frozen_without_dropout.pb",
"machinelearning.tensorflow.speech_2017-09-26/conv_actions_frozen.pb",
"apk-com.joytunes.simplypiano/jt_old_model_regression.pb",
"apk-com.joytunes.simplypiano/jt_extended_model_regression.pb",
]

# path = "apk-com.optimaxInvestments.GlassesUSA/mtcnn_freezed_model.pb"
flops_record = []
for path in paths:
    base_path = "/opt/Anonymous/workspace/mob-dl-rev/data/raw/tensorflow_model"
    model_path = "{}/{}".format(base_path, path)
    graph, graph_def = load_pb(model_path)

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd='op', options=opts)

    # return flops.total_float_ops

    # flops = tf.compat.v1.profiler.profile(graph, options = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation())
    # print('FLOP after freezing', flops.total_float_ops)
    flops_record.append(flops.total_float_ops)
    # typeset = set()
    # param_count = 0
    # for op in graph.get_operations():
    #     typeset.add(op.type)
    #     if op.type == "Const":
    #         param_count += sum([np.product(o.shape) for o in op.outputs])
            
    # # print(typeset)
    # print(param_count)

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated

Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.



=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

In [4]:
for x in flops_record:
    print(x)

0
814368072
228502
330227


In [ ]:
from src.advtool.attack import AdvTool
from src.advtool.WhiteBoxAttacks.config import AdvAttackConfig

config = AdvAttackConfig()
config.clip_range = (-1, 0.9921875)
config.eps = 8. / 255. * (config.clip_range[1] - config.clip_range[0])
# config.eps = 0.02
config.nb_iter = 100
pgd_attack = AdvTool.getWhiteBoxAttack(attack_name="pgd", config=config)

In [ ]:
model = (graph, graph_def, input_tensor, output_tensor)
adv_x = pgd_attack.run(model, x)

In [ ]:
out = sess.run(output_tensor, feed_dict={input_tensor: adv_x})
print(out)

In [ ]:
display(Image.fromarray((adv_x[0]*128 + 128).astype(np.uint8)))
# display(Image.fromarray((x[0]*128 + 128).astype(np.uint8)))

In [ ]:
img = Image.fromarray((adv_x[0]*128 + 128).astype(np.uint8))
img = np.asarray(img).astype(np.float32)
x = img.reshape(1, 224, 224, 3)
x -= 128.
x /= 128.
out = sess.run(output_tensor, feed_dict={input_tensor: x})
print(out)

In [ ]:
# Image.fromarray((adv_x[0]*128 + 128).astype(np.uint8)).save("adv_hotdog_3.bmp", format="bmp")
Image.fromarray((x[0]*128 + 128).astype(np.uint8)).save("hotdog_3.bmp", format="bmp")